In [8]:
!pip install --upgrade pip

# 라이브러리
import numpy as np
import pandas as pd
import datetime
import fsspec
import seaborn as sns
from IPython.lib.clipboard import osx_clipboard_get
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from time import time
import os
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pandas import DataFrame
import xgboost as xgb
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\wkdal\DREW_PY\.venv\Scripts\python.exe -m pip install --upgrade pip
C:\Users\wkdal\DREW_PY\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


In [9]:
# import dataset

store = pd.read_csv("C://Users//wkdal//DREW_PY//2025//Personal_Projects//store.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//wkdal//DREW_PY//2025//Personal_Projects//store.csv'